In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
import openai
import PyPDF2


# Download necessary resources (run once)
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

                         


In [2]:
# Extract Text from PDF

path = r"Amicus Brief on Behalf of Mississippi, Alabama, Alaska, Arkansas etc....pdf"
def extract_text(path):
    with open(path, 'rb') as file:
        text = ''
        pdf_read = PyPDF2.PdfReader(file)
        for page in range(len(pdf_read.pages)):
            text += pdf_read.pages[page].extract_text()
    return text;

# Storing text 
doc_text = extract_text(path)

In [3]:
# Removing stop words and punctuations as they're not necessary

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    # Remove punctuation
    text = ''.join([c for c in text if c not in punctuation])
    # Remove stopwords
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

doc_text = preprocess_text(doc_text)

In [4]:
print(doc_text)

UNITED STATES DISTRICT COURT NORTHERN DISTRICT TEXAS AMARILLO DIVISION ALLIANCE HIPPOCRATIC MEDICINE et al Plaintiff v Case 222cv00223 Z US FOOD DRUG ADMINISTRATION et al Defendants AMICUS CURIAE BRIEF MISSISSIPPI ALABAMA ALASKA ARKANSAS FLORIDA GEORGIA IDAHO INDIANA IOWA KANSAS KENTUCKY LOUISIANA MONTANA NEBRASKA OHIO OKLAHOMA SOUTH CAROLINA SOUTH DAKOTA TENNESSEE TEXAS UTAH WYOMING SUPPORT PLAINTIFFS ’ MOTION PRELIMINARY INJUNCTION Case 222cv00223Z Document 100 Filed 021323 Page 1 26 PageID 3760i TABLE CONTENTS Page TABLE AUTHORITIES ii INTRODUCTION INTEREST AMICI CURIAE SUMMARY ARGUMENT 1 BACKGROUND 4 ARGUMENT 7 Public Interest Equities Support Injunctive Relief FDA’s Actions Mifepristone 7 Public Interest Equities Weigh Strongly FDA’s Actions Actions Defy Federal Law 8 B FDA’s Actions Undermine Public Interest Determinations States—Not Federal Agencies —Are Entitled Make 11 C FDA’s Actions Harm Public Interest Undermining States’ Ability Protect Citizens Forcing States Divert Scarc

In [5]:
# Divinding text into smaller chunks 
# (so as we can use model to summerize these chunks of data)
def divide_text(text, chunk_size=200):
    words = text.split()
    num_words = len(words)
    divided_text = []
    start = 0

    while start < num_words:
        end = min(start + chunk_size, num_words)
        chunk = ' '.join(words[start:end])
        divided_text.append(chunk)
        start = end

    return divided_text

div_text = divide_text(doc_text)



In [12]:
# Firstly get the summary of divided chunks


def summarize_text(text):
    openai.api_key = 'Your API_KEY'
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt="Summarize the following text:\n\n" + text,
        max_tokens=100,
        temperature=0.1,
        top_p=1.0,
        
    )

    summary = response.choices[0].text.strip()
    return summary

summarized_chunks = []


# Concatinating all the chunks of summerized information
for text in div_text:
    temp = summarize_text(text)
    summarized_chunks.append(temp)


In [13]:
# Merge summary in one file
with open("summaries.txt", "w", encoding="utf-8") as file:
    for text in summarized_chunks:
        file.write(text)

    

In [20]:
import openai

# Set up OpenAI API key
openai.api_key = 'Your API_KEY'  # Replace with your OpenAI API key

# Read the summarized text file
file_path = "summaries.txt"

with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

    
# Define the prompt
prompt = f"""
Input: Provide 10 important points for the court legal document below: {text}
"""



# Generate response using OpenAI's completions API
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt=prompt,
  temperature=0.3,
  max_tokens=1000,
  stop=None,
  
)

# Extract the generated important points from the response
important_points = response.choices[0].text.strip().split("\n")

# Print the important points
print("Important Points:")
for point in important_points:
    print("- " + point)


Important Points:
- 1. The Biden Administration has taken multiple actions to protect access to reproductive healthcare services, including Executive Order 14076, regulations 21 CFR § 3142-1314, 21 CFR § 314500-4, 8-10, 13-14, 21 CFR § 314520-4, 21 CFR § 314500-314560, and the New Drug Antibiotic Biological Drug Product Regulations Accelerated Approval. 
- 2. The Supreme Court held last year that the authority to regulate or prohibit abortion is entrusted to the people and their elected representatives. 
- 3. The Biden Administration has been attempting to impose a nationwide policy for elective abortion drugs, but has been met with legal issues. 
- 4. The US Food and Drug Administration has been trying to establish a regime for access to chemical abortion drugs for two decades, disregarding safety measures and rolling back requirements. 
- 5. The FDA is responsible for approving new drugs and ensuring they are safe and effective. 
- 6. In 2011, the FDA established a Risk Evaluation Mi